In [1]:
from subprocess import call
from glob import glob
from nltk.corpus import stopwords
import os, struct
from tensorflow.core.example import example_pb2
import pyrouge
import shutil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import *

In [2]:
# cmd = '/root/miniconda2/bin/python run_summarization.py --mode=decode --single_pass=1 --coverage=True --vocab_path=finished_files/vocab --log_root=log --exp_name=myexperiment --data_path=test/temp_file --max_enc_steps=4000'
# generated_path = '/gttp/pointer-generator-tal/log/myexperiment/decode_test_4000maxenc_4beam_35mindec_100maxdec_ckpt-238410/'
# cmd = cmd.split()

vocab_path = '../data/DMQA/finished_files/vocab'
log_root = 'log'
exp_name = 'myexperiment'
data_path= 'test/temp_file'
max_enc_steps = 4000

cmd = ['python',
       'run_summarization.py',
       '--mode=decode',
       '--single_pass=1',
       '--coverage=True',
       '--vocab_path=' + vocab_path,
       '--log_root=' + log_root,
       '--exp_name=' + exp_name,
       '--data_path=' + data_path,
       '--max_enc_steps=' + str(max_enc_steps)]

generated_path = 'log/myexperiment/decode_test_4000maxenc_4beam_35mindec_100maxdec_ckpt-238410/'
max_len = 250
stopwords = set(stopwords.words('english'))
stemmer = PorterStemmer()

In [3]:
def pp(string):
    return ' '.join([stemmer.stem(word.decode('utf8')) for word in string.lower().split() if not word in stopwords])
    
def write_to_file(article, abstract, rel, writer):
    abstract = '<s> '+' '.join(abstract)+' </s>'
    #abstract = abstract.encode('utf8', 'ignore')
    #rel = rel.encode('utf8', 'ignore')
    #article = article.encode('utf8', 'ignore')
    tf_example = example_pb2.Example()
    tf_example.features.feature['abstract'].bytes_list.value.extend([bytes(abstract)])
    tf_example.features.feature['relevancy'].bytes_list.value.extend([bytes(rel)])
    tf_example.features.feature['article'].bytes_list.value.extend([bytes(article)])
    tf_example_str = tf_example.SerializeToString()
    str_len = len(tf_example_str)
    writer.write(struct.pack('q', str_len))
    writer.write(struct.pack('%ds' % str_len, tf_example_str))


def duck_iterator(i):
    duc_folder = 'duc0' + str(i) + 'tokenized/'
    for topic in os.listdir(duc_folder + 'testdata/docs/'):
        topic_folder = duc_folder + 'testdata/docs/' + topic
        if not os.path.isdir(topic_folder):
            continue
        query = ' '.join(open(duc_folder + 'queries/' + topic).readlines())
        model_files = glob(duc_folder + 'models/' + topic[:-1].upper() + '.*')

        topic_texts = [' '.join(open(topic_folder + '/' + file).readlines()).replace('\n', '') for file in
                       os.listdir(topic_folder)]

        abstracts = [' '.join(open(f).readlines()) for f in model_files]
        yield topic_texts, abstracts, query


def count_score(sent, ref):
    ref = pp(ref).split()
    sent = ' '.join(pp(w) for w in sent.lower().split() if not w in stopwords)
    return sum([1. if w in ref else 0. for w in sent.split()])


def get_tfidf_score_func(magic = 1):
    corpus = []
    for i in range(5, 8):
        for topic_texts, _, _ in duck_iterator(i):
            corpus += [pp(t) for t in topic_texts]

    vectorizer = TfidfVectorizer()
    vectorizer.fit_transform(corpus)

    def tfidf_score_func(sent, ref):
        ref = [pp(s) for s in ref.split(' . ')]
        sent = pp(sent)
        v1 = vectorizer.transform([sent])
        v2s = [vectorizer.transform([r]) for r in ref]
        return max([cosine_similarity(v1, v2)[0][0] for v2 in v2s])
    return tfidf_score_func

In [4]:
class Summary:
    def __init__(self, texts, abstracts, query):
        
        #texts = sorted([(tfidf_score(query, text), text) for text in texts], reverse=True)
        texts = sorted([(count_score(text, ' '.join(abstracts)), text) for text in texts], reverse=True)

        texts = [text[1] for text in texts]
        self.texts = texts
        self.abstracts = abstracts
        self.query = query
        self.summary = []
        self.words = set()
        self.length = 0

    def most_similar(self, sent, text):
        return max([(count_score(s, sent), s) for s in text])[1]

    def add_sum(self, summ):
        text = self.texts.pop(0).split(' . ')
        if len(self.texts) == 0: return True
        found_sents = []
        for sent in summ:
            ms = self.most_similar(sent, text)
            if ms in found_sents:
                continue
            found_sents.append(sent)
            splitted = pp(sent).split()
            length = len(splitted) 
            splitted = set(splitted)       
            if self.length+length > max_len: return True
            if len(splitted - self.words) < int(len(splitted)*0.5): return False
            self.words |= splitted     
            self.summary.append(sent)
            self.length +=length
        return False

    def get(self):
        text = self.texts[0]
        sents = text.split(' . ')
        #score_per_sent = [(count_score(sent, self.query), sent) for sent in sents]
        score_per_sent = [(count_score(sent, ' '.join(self.abstracts)), sent) for sent in sents]

        scores = []
        for score, sent in score_per_sent:
            scores += [score] * (len(sent.split()) + 1)
        scores = str(scores[:-1])
        return text, 'a', scores

def get_summaries(path):
    path = path+'decoded/'
    out = {}
    for file_name in os.listdir(path):
        index = int(file_name.split('_')[0])
        out[index] = open(path+file_name).readlines()
    return out

def rouge_eval(ref_dir, dec_dir):
    """Evaluate the files in ref_dir and dec_dir with pyrouge, returning results_dict"""
    r = pyrouge.Rouge155()
    r.model_filename_pattern = '#ID#_reference_(\d+).txt'
    r.system_filename_pattern = '(\d+)_decoded.txt'
    r.model_dir = ref_dir
    r.system_dir = dec_dir
    return r.convert_and_evaluate()

def evaluate(summaries):
    for path in ['eval/ref', 'eval/dec']:
        if os.path.exists(path): shutil.rmtree(path, True)
        os.mkdir(path)
    for i, summ in enumerate(summaries):
        for j,abs in enumerate(summ.abstracts):
            with open('eval/ref/'+str(i)+'_reference_'+str(j)+'.txt', 'w') as f:
                f.write(abs)
        with open('eval/dec/'+str(i)+'_decoded.txt', 'w') as f:
            f.write(' '.join(summ.summary))
    print rouge_eval('eval/ref/', 'eval/dec/') 

In [5]:
for i in [5]:
    duc_num = i
    done_summaries = []
    summaries = [Summary(texts, abstracts, query) for texts, abstracts, query in duck_iterator(i)]
    while summaries:
        with open('test/temp_file', 'wb') as writer:
            for summ in summaries:
                article, abstract, scores = summ.get()
                write_to_file(article, abstracts, scores, writer)
        call(['rm', '-r', generated_path])     
        call(cmd)
        generated_summaries = get_summaries(generated_path)
        should_remove = []
        for i in range(len(summaries)):
            if summaries[i].add_sum(generated_summaries[i]):
                should_remove.append(i)
        for i in should_remove[::-1]: 
            done_summaries.append(summaries.pop(i))
    evaluate(done_summaries)
    print duc_num

2019-02-12 10:56:16,313 [MainThread  ] [INFO ]  Writing summaries.
2019-02-12 10:56:16,317 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmpNSYAgZ/system and model files to /tmp/tmpNSYAgZ/model.
2019-02-12 10:56:16,318 [MainThread  ] [INFO ]  Processing files in eval/dec/.
2019-02-12 10:56:16,319 [MainThread  ] [INFO ]  Processing 0_decoded.txt.
2019-02-12 10:56:16,322 [MainThread  ] [INFO ]  Processing 1_decoded.txt.
2019-02-12 10:56:16,324 [MainThread  ] [INFO ]  Processing 2_decoded.txt.
2019-02-12 10:56:16,326 [MainThread  ] [INFO ]  Processing 3_decoded.txt.
2019-02-12 10:56:16,328 [MainThread  ] [INFO ]  Processing 4_decoded.txt.
2019-02-12 10:56:16,330 [MainThread  ] [INFO ]  Processing 5_decoded.txt.
2019-02-12 10:56:16,333 [MainThread  ] [INFO ]  Processing 6_decoded.txt.
2019-02-12 10:56:16,335 [MainThread  ] [INFO ]  Processing 7_decoded.txt.
2019-02-12 10:56:16,337 [MainThread  ] [INFO ]  Processing 8_decoded.txt.
2019-02-12 10:56:16,339 [MainThr

2019-02-12 10:56:16,542 [MainThread  ] [INFO ]  Processing 7_reference_8.txt.
2019-02-12 10:56:16,544 [MainThread  ] [INFO ]  Processing 8_reference_0.txt.
2019-02-12 10:56:16,546 [MainThread  ] [INFO ]  Processing 8_reference_1.txt.
2019-02-12 10:56:16,549 [MainThread  ] [INFO ]  Processing 8_reference_2.txt.
2019-02-12 10:56:16,551 [MainThread  ] [INFO ]  Processing 8_reference_3.txt.
2019-02-12 10:56:16,553 [MainThread  ] [INFO ]  Processing 9_reference_0.txt.
2019-02-12 10:56:16,555 [MainThread  ] [INFO ]  Processing 9_reference_1.txt.
2019-02-12 10:56:16,557 [MainThread  ] [INFO ]  Processing 9_reference_2.txt.
2019-02-12 10:56:16,559 [MainThread  ] [INFO ]  Processing 9_reference_3.txt.
2019-02-12 10:56:16,562 [MainThread  ] [INFO ]  Processing 9_reference_4.txt.
2019-02-12 10:56:16,564 [MainThread  ] [INFO ]  Processing 9_reference_5.txt.
2019-02-12 10:56:16,566 [MainThread  ] [INFO ]  Processing 9_reference_6.txt.
2019-02-12 10:56:16,569 [MainThread  ] [INFO ]  Processing 9_ref

2019-02-12 10:56:16,771 [MainThread  ] [INFO ]  Processing 23_reference_3.txt.
2019-02-12 10:56:16,773 [MainThread  ] [INFO ]  Processing 24_reference_0.txt.
2019-02-12 10:56:16,776 [MainThread  ] [INFO ]  Processing 24_reference_1.txt.
2019-02-12 10:56:16,778 [MainThread  ] [INFO ]  Processing 24_reference_2.txt.
2019-02-12 10:56:16,780 [MainThread  ] [INFO ]  Processing 24_reference_3.txt.
2019-02-12 10:56:16,782 [MainThread  ] [INFO ]  Processing 25_reference_0.txt.
2019-02-12 10:56:16,784 [MainThread  ] [INFO ]  Processing 25_reference_1.txt.
2019-02-12 10:56:16,787 [MainThread  ] [INFO ]  Processing 25_reference_2.txt.
2019-02-12 10:56:16,789 [MainThread  ] [INFO ]  Processing 25_reference_3.txt.
2019-02-12 10:56:16,791 [MainThread  ] [INFO ]  Processing 26_reference_0.txt.
2019-02-12 10:56:16,794 [MainThread  ] [INFO ]  Processing 26_reference_1.txt.
2019-02-12 10:56:16,796 [MainThread  ] [INFO ]  Processing 26_reference_2.txt.
2019-02-12 10:56:16,798 [MainThread  ] [INFO ]  Proc

2019-02-12 10:56:17,001 [MainThread  ] [INFO ]  Processing 42_reference_1.txt.
2019-02-12 10:56:17,003 [MainThread  ] [INFO ]  Processing 42_reference_2.txt.
2019-02-12 10:56:17,005 [MainThread  ] [INFO ]  Processing 42_reference_3.txt.
2019-02-12 10:56:17,007 [MainThread  ] [INFO ]  Processing 43_reference_0.txt.
2019-02-12 10:56:17,009 [MainThread  ] [INFO ]  Processing 43_reference_1.txt.
2019-02-12 10:56:17,012 [MainThread  ] [INFO ]  Processing 43_reference_2.txt.
2019-02-12 10:56:17,014 [MainThread  ] [INFO ]  Processing 43_reference_3.txt.
2019-02-12 10:56:17,016 [MainThread  ] [INFO ]  Processing 44_reference_0.txt.
2019-02-12 10:56:17,018 [MainThread  ] [INFO ]  Processing 44_reference_1.txt.
2019-02-12 10:56:17,020 [MainThread  ] [INFO ]  Processing 44_reference_2.txt.
2019-02-12 10:56:17,023 [MainThread  ] [INFO ]  Processing 44_reference_3.txt.
2019-02-12 10:56:17,025 [MainThread  ] [INFO ]  Processing 45_reference_0.txt.
2019-02-12 10:56:17,027 [MainThread  ] [INFO ]  Proc

---------------------------------------------
1 ROUGE-1 Average_R: 0.38579 (95%-conf.int. 0.36853 - 0.40241)
1 ROUGE-1 Average_P: 0.31437 (95%-conf.int. 0.30199 - 0.32636)
1 ROUGE-1 Average_F: 0.34579 (95%-conf.int. 0.33154 - 0.35920)
---------------------------------------------
1 ROUGE-2 Average_R: 0.06499 (95%-conf.int. 0.05737 - 0.07312)
1 ROUGE-2 Average_P: 0.05266 (95%-conf.int. 0.04665 - 0.05853)
1 ROUGE-2 Average_F: 0.05808 (95%-conf.int. 0.05134 - 0.06483)
---------------------------------------------
1 ROUGE-3 Average_R: 0.01887 (95%-conf.int. 0.01441 - 0.02373)
1 ROUGE-3 Average_P: 0.01516 (95%-conf.int. 0.01175 - 0.01882)
1 ROUGE-3 Average_F: 0.01678 (95%-conf.int. 0.01292 - 0.02097)
---------------------------------------------
1 ROUGE-4 Average_R: 0.01010 (95%-conf.int. 0.00681 - 0.01368)
1 ROUGE-4 Average_P: 0.00802 (95%-conf.int. 0.00552 - 0.01073)
1 ROUGE-4 Average_F: 0.00892 (95%-conf.int. 0.00612 - 0.01199)
---------------------------------------------
1 ROUGE-L Aver